# Harmonic Regression Analysis with geeagri

This notebook demonstrates how to use the `HarmonicRegression` class in geeagri to analyze time series data from satellite imagery using Google Earth Engine.

## Setup and Imports
Install geeagri if needed and import required libraries.

In [ ]:
# !pip install geeagri
import ee
import geeagri
from geeagri.timeseries import HarmonicRegression
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
ee.Authenticate()
ee.Initialize()

## Select Region and Data
Define a point of interest and select an image collection and band for analysis.

In [ ]:
lat, lon = 37.0510, -120.3022  # Example: Central California
point = ee.Geometry.Point([lon, lat])

collection = ee.ImageCollection('MODIS/061/MOD13Q1')
band = 'NDVI'
start_date = '2015-01-01'
end_date = '2017-01-01'
scale = 250

## Extract Time Series
Extract the NDVI time series for the selected point.

In [ ]:
df = geeagri.extract_timeseries_to_point(
    lat, lon, collection, start_date, end_date, band_names=[band], scale=scale
)
df['NDVI'] = df['NDVI'] * 0.0001  # Apply scale factor for MODIS NDVI
df.head()

## Fit Harmonic Regression
Create a HarmonicRegression object and fit the model to the NDVI time series.

In [ ]:
ref_date = start_date
order = 2  # Number of harmonics
hr = HarmonicRegression(collection, ref_date, band, order=order)
harmonic_coeffs = hr.get_harmonic_coeffs()

## Visualize Fitted Harmonics
Compute and plot the fitted harmonic time series alongside the original NDVI data.

In [ ]:
fitted_coll = hr.get_fitted_harmonics(harmonic_coeffs)
# Extract fitted values at the point
fitted_list = fitted_coll.getRegion(point, scale=scale).getInfo()
fitted_df = pd.DataFrame(fitted_list[1:], columns=fitted_list[0])
fitted_df['time'] = pd.to_datetime(fitted_df['time'], unit='ms')
fitted_df = fitted_df[['time', 'fitted']].dropna()
# Merge with original NDVI
merged = pd.merge(df, fitted_df, on='time', how='inner')
plt.figure(figsize=(10,5))
plt.plot(merged['time'], merged['NDVI'], label='Observed NDVI', marker='o')
plt.plot(merged['time'], merged['fitted'], label='Fitted Harmonic', linestyle='--')
plt.xlabel('Date')
plt.ylabel('NDVI')
plt.title('Harmonic Regression Fit to NDVI Time Series')
plt.legend()
plt.show()

## Next Steps
- Try different harmonic orders for more/less complex fits.
- Use other bands or regions.
- Explore phase and amplitude extraction for phenology analysis.